In [2]:
import cv2
import numpy as np
import time

def main():
    #initialize video capture
    cap = cv2.VideoCapture(0)

    #create a background subtractor
    back_sub = cv2.createBackgroundSubtractorMOG2()

    #variables to manage screenshot capture
    object_in_center = False
    center_tolerance = 50 
    screenshot_taken = False

    while True:
        #read frame
        ret, frame = cap.read()
        if not ret:
            break

        #apply background subtraction
        fg_mask = back_sub.apply(frame)

        #find contours in the foreground mask
        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        #draw the contours on the original frame
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if cv2.contourArea(contour) > 500:  # Minimum contour area to avoid noise
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                #check if the object is in the center of the screen
                h, w, _ = frame.shape
                cx, cy = x + w // 2, y + h // 2
                center_x, center_y = w // 2, h // 2

                if (abs(cx - center_x) <= center_tolerance and
                    abs(cy - center_y) <= center_tolerance):
                    object_in_center = True
                    #only take a screenshot if it hasn't been taken yet
                    if not screenshot_taken:
                        #create a mask for the detected object
                        object_mask = np.zeros_like(frame)
                        cv2.drawContours(object_mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)
                        
                        #extract the object from the frame
                        object_image = cv2.bitwise_and(frame, object_mask)

                        #convert to grayscale
                        gray_image = cv2.cvtColor(object_image, cv2.COLOR_BGR2GRAY)

                        #remove background by applying mask
                        masked_gray_image = cv2.bitwise_and(gray_image, gray_image, mask=cv2.cvtColor(object_mask, cv2.COLOR_BGR2GRAY))

                        #save the screenshot
                        screenshot_filename = f'screenshot_{int(time.time())}.png'
                        cv2.imwrite(screenshot_filename, masked_gray_image)
                        print(f"Screenshot taken: {screenshot_filename}")

                        screenshot_taken = True  #mark that we've taken a photo
                else:
                    object_in_center = False
                    screenshot_taken = False  #allow screenshot to be taken again

        #display the original frame
        cv2.imshow('Frame', frame)

        #q to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    #close up program
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
